In [1]:

import os
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split



In [2]:

seq_length=2
x_data_dim=4
min_max_norm_flag=True


In [3]:

def data_sequence(input_data_X,input_data_Y, seq_length):
    dataX = []
    dataY = []
    dataX.clear()
    dataY.clear()
    for i in range(0, len(input_data_Y) - seq_length):
        _x = input_data_X[i:i + seq_length]
        _y = input_data_Y[i + seq_length]  # Next close price
        #print(_x, "->", _y)
        dataX.append(_x)
        dataY.append(_y)

    result_X = np.array(dataX)
    result_Y = np.array(dataY)
    return result_X, result_Y


In [4]:

data_dir = '../dataset'
fname = os.path.join(data_dir, 'data-02-stock_daily.csv')
df = pd.read_csv(fname)
dataset=df.copy()
ori_Y=dataset.pop("Close")
ori_X=dataset.copy()


In [5]:

X_train, X_test, Y_train, Y_test = train_test_split(ori_X,ori_Y, test_size=0.2, shuffle=False)
X_train, X_val, Y_train, Y_val= train_test_split(X_train,Y_train, test_size=0.2, shuffle=False)


In [6]:

## 데이터의 min , max, mean, std 값 구하기.
dataset_stats = X_train.describe()
dataset_stats = dataset_stats.transpose()

## data normalization
def min_max_norm(x):
  return (x - dataset_stats['min']) / (dataset_stats['max'] - dataset_stats['min'])

def standard_norm(x):
  return (x - dataset_stats['mean']) / dataset_stats['std']

if min_max_norm_flag == True:
    min_max_norm_train_data = min_max_norm(X_train)
    min_max_norm_val_data = min_max_norm(X_val)
    min_max_norm_test_data = min_max_norm(X_test)

    sequence_train_x, sequence_train_y =data_sequence(min_max_norm_train_data.values.tolist(), Y_train.values.tolist(), seq_length )
    sequence_val_x, sequence_val_y = data_sequence(min_max_norm_val_data.values.tolist(), Y_val.values.tolist(), seq_length )
    sequence_test_x, sequence_test_y =data_sequence(min_max_norm_test_data.values.tolist(), Y_test.values.tolist(), seq_length )
else:
    sequence_train_x, sequence_train_y = data_sequence(X_train.values.tolist(),Y_train.values.tolist(), seq_length)
    sequence_val_x, sequence_val_y = data_sequence(X_val.values.tolist(), Y_val.values.tolist(), seq_length)
    sequence_test_x, sequence_test_y = data_sequence(X_test.values.tolist(), Y_test.values.tolist(),seq_length)



In [7]:

## 모델
input_Layer = tf.keras.layers.Input(shape=(seq_length, x_data_dim))
x= tf.keras.layers.SimpleRNN(50,activation='tanh')(input_Layer) ## RNN
x=tf.keras.layers.Dense(100,activation='relu')(x)
x=tf.keras.layers.Dense(50,activation='relu')(x)
Out_Layer=tf.keras.layers.Dense(1,activation=None)(x)
model = tf.keras.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()



Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2, 4)]            0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 50)                2750      
_________________________________________________________________
dense (Dense)                (None, 100)               5100      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 12,951
Trainable params: 12,951
Non-trainable params: 0
_________________________________________________________________


In [8]:

loss_function=tf.keras.losses.mean_squared_error
optimize=tf.keras.optimizers.Adam(learning_rate=0.001)
metric=tf.keras.metrics.mean_absolute_error
model.compile(loss=loss_function,
              optimizer=optimize,
              metrics=[metric])

history = model.fit(sequence_train_x, sequence_train_y, epochs=500, batch_size=100, validation_data=(sequence_val_x,sequence_val_y))
print(model.evaluate(sequence_test_x, sequence_test_y))


Epoch 1/500
5/5 [==============================] - 2s 400ms/step - loss: 500052.5625 - mean_absolute_error: 702.0932 - val_loss: 287999.7188 - val_mean_absolute_error: 536.3427
Epoch 2/500
5/5 [==============================] - 0s 43ms/step - loss: 498936.0938 - mean_absolute_error: 701.3300 - val_loss: 287738.0625 - val_mean_absolute_error: 536.1008
Epoch 3/500
5/5 [==============================] - 0s 45ms/step - loss: 497450.6562 - mean_absolute_error: 700.3093 - val_loss: 287337.6250 - val_mean_absolute_error: 535.7307
Epoch 4/500
5/5 [==============================] - 0s 38ms/step - loss: 495230.8438 - mean_absolute_error: 698.7808 - val_loss: 286701.0312 - val_mean_absolute_error: 535.1417
Epoch 5/500
5/5 [==============================] - 0s 46ms/step - loss: 491795.2188 - mean_absolute_error: 696.4082 - val_loss: 285683.7812 - val_mean_absolute_error: 534.1986
Epoch 6/500
5/5 [==============================] - 0s 41ms/step - loss: 486817.3125 - mean_absolute_error: 692.9438 - v

5/5 [==============================] - 0s 40ms/step - loss: 2117.0020 - mean_absolute_error: 37.1339 - val_loss: 2189.1360 - val_mean_absolute_error: 35.0514
Epoch 49/500
5/5 [==============================] - 0s 69ms/step - loss: 2101.8684 - mean_absolute_error: 37.0295 - val_loss: 2208.1487 - val_mean_absolute_error: 35.0117
Epoch 50/500
5/5 [==============================] - 0s 75ms/step - loss: 2088.7905 - mean_absolute_error: 36.9433 - val_loss: 2225.6301 - val_mean_absolute_error: 35.0042
Epoch 51/500
5/5 [==============================] - 0s 93ms/step - loss: 2075.0713 - mean_absolute_error: 36.8819 - val_loss: 2249.2407 - val_mean_absolute_error: 35.0609
Epoch 52/500
5/5 [==============================] - 1s 104ms/step - loss: 2063.8818 - mean_absolute_error: 36.8615 - val_loss: 2273.2266 - val_mean_absolute_error: 35.2136
Epoch 53/500
5/5 [==============================] - 0s 75ms/step - loss: 2049.7385 - mean_absolute_error: 36.7857 - val_loss: 2281.0183 - val_mean_absolute_e

Epoch 96/500
5/5 [==============================] - 0s 90ms/step - loss: 1292.8301 - mean_absolute_error: 29.7176 - val_loss: 1472.1361 - val_mean_absolute_error: 29.5663
Epoch 97/500
5/5 [==============================] - 0s 60ms/step - loss: 1270.7841 - mean_absolute_error: 29.4684 - val_loss: 1441.5555 - val_mean_absolute_error: 29.2785
Epoch 98/500
5/5 [==============================] - 0s 79ms/step - loss: 1246.7848 - mean_absolute_error: 29.1882 - val_loss: 1420.5709 - val_mean_absolute_error: 29.1006
Epoch 99/500
5/5 [==============================] - 0s 59ms/step - loss: 1224.4498 - mean_absolute_error: 28.9331 - val_loss: 1397.4204 - val_mean_absolute_error: 28.8907
Epoch 100/500
5/5 [==============================] - 0s 58ms/step - loss: 1201.0779 - mean_absolute_error: 28.6599 - val_loss: 1330.1055 - val_mean_absolute_error: 28.1326
Epoch 101/500
5/5 [==============================] - 0s 66ms/step - loss: 1176.5717 - mean_absolute_error: 28.3522 - val_loss: 1308.9680 - val_m

Epoch 144/500
5/5 [==============================] - 1s 108ms/step - loss: 494.0660 - mean_absolute_error: 18.3620 - val_loss: 685.1170 - val_mean_absolute_error: 19.3160
Epoch 145/500
5/5 [==============================] - 0s 99ms/step - loss: 486.4335 - mean_absolute_error: 18.2081 - val_loss: 690.0527 - val_mean_absolute_error: 19.4214
Epoch 146/500
5/5 [==============================] - 0s 62ms/step - loss: 479.9884 - mean_absolute_error: 18.0607 - val_loss: 662.7659 - val_mean_absolute_error: 18.9331
Epoch 147/500
5/5 [==============================] - 0s 74ms/step - loss: 475.2971 - mean_absolute_error: 17.9543 - val_loss: 646.6312 - val_mean_absolute_error: 18.6291
Epoch 148/500
5/5 [==============================] - 0s 63ms/step - loss: 469.9846 - mean_absolute_error: 17.8129 - val_loss: 673.0884 - val_mean_absolute_error: 19.1679
Epoch 149/500
5/5 [==============================] - 0s 95ms/step - loss: 460.4382 - mean_absolute_error: 17.6431 - val_loss: 632.5696 - val_mean_abs

5/5 [==============================] - 1s 194ms/step - loss: 290.0945 - mean_absolute_error: 13.7077 - val_loss: 320.9053 - val_mean_absolute_error: 13.3274
Epoch 193/500
5/5 [==============================] - 0s 100ms/step - loss: 289.9375 - mean_absolute_error: 13.6766 - val_loss: 307.0463 - val_mean_absolute_error: 12.9958
Epoch 194/500
5/5 [==============================] - 0s 78ms/step - loss: 285.3521 - mean_absolute_error: 13.5874 - val_loss: 325.2544 - val_mean_absolute_error: 13.5456
Epoch 195/500
5/5 [==============================] - 0s 83ms/step - loss: 281.2289 - mean_absolute_error: 13.4897 - val_loss: 288.7626 - val_mean_absolute_error: 12.5530
Epoch 196/500
5/5 [==============================] - 0s 58ms/step - loss: 281.4857 - mean_absolute_error: 13.4770 - val_loss: 307.1467 - val_mean_absolute_error: 13.1214
Epoch 197/500
5/5 [==============================] - 1s 123ms/step - loss: 276.7816 - mean_absolute_error: 13.3833 - val_loss: 303.1021 - val_mean_absolute_error:

...

Epoch 289/500
5/5 [==============================] - 0s 72ms/step - loss: 107.8201 - mean_absolute_error: 7.6455 - val_loss: 49.2085 - val_mean_absolute_error: 5.5833
Epoch 290/500
5/5 [==============================] - 0s 56ms/step - loss: 106.0840 - mean_absolute_error: 7.5925 - val_loss: 52.9730 - val_mean_absolute_error: 5.8166
Epoch 291/500
5/5 [==============================] - 0s 65ms/step - loss: 106.6867 - mean_absolute_error: 7.5663 - val_loss: 42.5861 - val_mean_absolute_error: 5.1819
Epoch 292/500
5/5 [==============================] - 0s 51ms/step - loss: 103.7475 - mean_absolute_error: 7.4425 - val_loss: 42.7379 - val_mean_absolute_error: 5.2021
Epoch 293/500
5/5 [==============================] - 0s 53ms/step - loss: 103.2394 - mean_absolute_error: 7.3886 - val_loss: 37.2876 - val_mean_absolute_error: 4.8074
Epoch 294/500
5/5 [==============================] - 0s 72ms/step - loss: 102.7691 - mean_absolute_error: 7.3995 - val_loss: 43.3868 - val_mean_absolute_error: 5.266

5/5 [==============================] - 0s 67ms/step - loss: 75.2451 - mean_absolute_error: 5.8401 - val_loss: 18.8506 - val_mean_absolute_error: 3.3558
Epoch 339/500
5/5 [==============================] - 0s 68ms/step - loss: 74.9282 - mean_absolute_error: 5.7929 - val_loss: 19.0331 - val_mean_absolute_error: 3.3752
Epoch 340/500
5/5 [==============================] - 0s 83ms/step - loss: 74.9114 - mean_absolute_error: 5.8061 - val_loss: 19.2855 - val_mean_absolute_error: 3.3955
Epoch 341/500
5/5 [==============================] - 0s 53ms/step - loss: 74.0869 - mean_absolute_error: 5.7510 - val_loss: 18.7494 - val_mean_absolute_error: 3.3505
Epoch 342/500
5/5 [==============================] - 0s 74ms/step - loss: 74.7601 - mean_absolute_error: 5.7837 - val_loss: 18.6389 - val_mean_absolute_error: 3.3360
Epoch 343/500
5/5 [==============================] - 0s 79ms/step - loss: 74.5621 - mean_absolute_error: 5.7987 - val_loss: 18.4443 - val_mean_absolute_error: 3.3175
Epoch 344/500
5/5 

5/5 [==============================] - 0s 72ms/step - loss: 67.3590 - mean_absolute_error: 5.4086 - val_loss: 17.8161 - val_mean_absolute_error: 3.2515
Epoch 388/500
5/5 [==============================] - 0s 55ms/step - loss: 67.6846 - mean_absolute_error: 5.3390 - val_loss: 16.5231 - val_mean_absolute_error: 3.1755
Epoch 389/500
5/5 [==============================] - 1s 118ms/step - loss: 67.9425 - mean_absolute_error: 5.4406 - val_loss: 19.9373 - val_mean_absolute_error: 3.4763
Epoch 390/500
5/5 [==============================] - 1s 130ms/step - loss: 65.9437 - mean_absolute_error: 5.2687 - val_loss: 16.2723 - val_mean_absolute_error: 3.1305
Epoch 391/500
5/5 [==============================] - 0s 78ms/step - loss: 68.8370 - mean_absolute_error: 5.4032 - val_loss: 17.2824 - val_mean_absolute_error: 3.1918
Epoch 392/500
5/5 [==============================] - 0s 64ms/step - loss: 66.3276 - mean_absolute_error: 5.3714 - val_loss: 16.9865 - val_mean_absolute_error: 3.1764
Epoch 393/500
5/

5/5 [==============================] - 0s 61ms/step - loss: 61.4623 - mean_absolute_error: 5.0474 - val_loss: 16.7060 - val_mean_absolute_error: 3.2516
Epoch 437/500
5/5 [==============================] - 0s 55ms/step - loss: 67.1858 - mean_absolute_error: 5.3748 - val_loss: 16.7485 - val_mean_absolute_error: 3.1619
Epoch 438/500
5/5 [==============================] - 0s 59ms/step - loss: 65.5172 - mean_absolute_error: 5.2838 - val_loss: 17.0511 - val_mean_absolute_error: 3.1952
Epoch 439/500
5/5 [==============================] - 0s 43ms/step - loss: 62.5078 - mean_absolute_error: 5.0293 - val_loss: 15.4134 - val_mean_absolute_error: 3.0926
Epoch 440/500
5/5 [==============================] - 0s 48ms/step - loss: 60.4211 - mean_absolute_error: 4.9932 - val_loss: 18.5293 - val_mean_absolute_error: 3.3611
Epoch 441/500
5/5 [==============================] - 0s 80ms/step - loss: 61.0769 - mean_absolute_error: 5.0796 - val_loss: 15.9685 - val_mean_absolute_error: 3.0790
Epoch 442/500
5/5 

Epoch 485/500
5/5 [==============================] - 0s 72ms/step - loss: 60.5146 - mean_absolute_error: 4.8925 - val_loss: 15.3137 - val_mean_absolute_error: 3.0107
Epoch 486/500
5/5 [==============================] - 0s 66ms/step - loss: 59.4142 - mean_absolute_error: 4.9404 - val_loss: 19.8678 - val_mean_absolute_error: 3.5031
Epoch 487/500
5/5 [==============================] - 0s 63ms/step - loss: 59.8270 - mean_absolute_error: 4.9288 - val_loss: 15.0806 - val_mean_absolute_error: 3.0132
Epoch 488/500
5/5 [==============================] - 0s 58ms/step - loss: 59.6036 - mean_absolute_error: 4.9166 - val_loss: 15.0760 - val_mean_absolute_error: 3.0443
Epoch 489/500
5/5 [==============================] - 0s 45ms/step - loss: 58.1549 - mean_absolute_error: 4.8410 - val_loss: 14.8800 - val_mean_absolute_error: 2.9833
Epoch 490/500
5/5 [==============================] - 0s 74ms/step - loss: 58.6122 - mean_absolute_error: 4.8405 - val_loss: 16.9083 - val_mean_absolute_error: 3.1782
Epoc

In [ ]:

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

prediction_Y=model.predict(sequence_test_x).flatten()

visual_y=[]
visual_pre_y=[]
for i in range(len(prediction_Y)):
    label = sequence_test_y[i]
    prediction = prediction_Y[i]
    print("실제가격: {:.3f}, 예상가격: {:.3f}".format(label, prediction))
    visual_y.append(label)
    visual_pre_y.append(prediction)

time = range(1, len(visual_y) + 1)
plt.plot(time, visual_y, 'r', label='ture')
plt.plot(time, visual_pre_y, 'b', label='prediction')
plt.title('stock prediction')
plt.xlabel('time')
plt.ylabel('value')
plt.legend()
plt.show()